In [321]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.datasets import load_svmlight_file
from datetime import timedelta


## Overview
The baseline model is trained using the following features:

1) Diagnosis (value 1 for every diagnosis)

2) Lab_results (value is the mean of recorded values, scaled by maximum

In [322]:
### Sampling on patients
path = "../../data/"
patients = pd.read_csv(f"{path}PATIENTS.csv")
#ensuring every patient is unique
print(f"{len(patients.SUBJECT_ID.unique())} unique patients in {len(patients)} rows")
#sampling random patients
patients_sample = patients.sample(n = 10000, random_state= 1)
# patients

46520 unique patients in 46520 rows


In [323]:
#get admissions
admissions = pd.read_csv(f"{path}ADMISSIONS.csv.gz")
admissions = pd.merge(patients_sample.SUBJECT_ID,admissions)
#admissions

In [324]:
# Global Constants
OBSERVATION_WINDOW = 365*2
PREDICTION_WINDOW = 30
USEFUL_COLUMNS = ['SUBJECT_ID','FEATURE','DATE','VALUE']

In [325]:
# note_events = pd.read_csv("../data/NOTEEVENTS.csv")
# #sample and save for these 1000 patients
# note_events_sample = pd.merge(patients_sample.SUBJECT_ID,note_events)
# note_events_sample.to_csv("../data/NOTEEVENTS_SAMPLE.csv")

In [326]:
### Incomplete: Using AWS glue, for now I will do stuff locally
# import boto3
# Creating the low level functional client
# client = boto3.client(
#     'glue',
#     aws_access_key_id = '',
#     aws_secret_access_key = '',
#     region_name = 'us-east-1'
# )
# clientResponse = client.get_table(DatabaseName="mimiciii",Name="admissions")

In [327]:
def convert_icd9(icd9_object):
    """
    :param icd9_object: ICD-9 code (Pandas/Numpy object).
    :return: extracted main digits of ICD-9 code
    """
    icd9_str = str(icd9_object)

    if icd9_str[0] == 'E': #if code starts with E
        converted = icd9_str[:4]
    else: #if they start with V or numeric
        converted = icd9_str[:3]

    return converted

def build_codemap(dataset):
    """
    :return: Dict of code map {main-digits of ICD9: unique feature ID}
    """
    # TODO: We build a code map using ONLY train data. Think about how to construct validation/test sets using this.
    df_digits = dataset['FEATURE'].unique()
    codemap = {}
    for i in range(0,len(df_digits)):
        codemap[df_digits[i]] = i


    return codemap

# #dataset that contains HADM_ID
# def filter_by_time_window(dataset,admissions,patients):
#     dataset = pd.merge(dataset,admissions[['HADM_ID','ADMITTIME']],on = ['HADM_ID'])
#     dataset = pd.merge(dataset,patients[['SUBJECT_ID','DOD']])
#     dataset.loc[]
    

### Adding diagnoses data

In [328]:
#read and etl on diagnoses
diagnoses = pd.read_csv(f"{path}DIAGNOSES_ICD.csv.gz")
# sample for the patients
diagnoses = pd.merge(patients_sample.SUBJECT_ID,diagnoses)

#add admit time to diagnosis 
diagnoses = diagnoses.merge(admissions[['HADM_ID','ADMITTIME']],on = 'HADM_ID')

#convert features
diagnoses['VALUE'] = 1
diagnoses["ICD9_CODE"] = diagnoses["ICD9_CODE"].apply(convert_icd9).apply(lambda x: f"DIAG_{x}")
diagnoses = diagnoses.rename(columns = {'ICD9_CODE':'FEATURE',"ADMITTIME":'DATE'})
diagnoses = diagnoses[USEFUL_COLUMNS]

diagnoses.drop_duplicates(inplace = True)


In [329]:
diagnoses

,SUBJECT_ID,FEATURE,DATE,VALUE
0,4074,DIAG_038,2204-02-04 07:26:00,1
1,4074,DIAG_785,2204-02-04 07:26:00,1
2,4074,DIAG_578,2204-02-04 07:26:00,1
3,4074,DIAG_427,2204-02-04 07:26:00,1
4,4074,DIAG_428,2204-02-04 07:26:00,1
...,...,...,...,...
139065,62212,DIAG_790,2176-06-09 13:01:00,1
139066,62212,DIAG_781,2176-06-09 13:01:00,1
139067,62212,DIAG_338,2176-06-09 13:01:00,1
139068,62212,DIAG_V15,2176-06-09 13:01:00,1


### Lab Results

In [330]:

#read and etl on lab_results
# lab_results = pd.read_csv(f"{path}LABEVENTS_SAMPLE.csv")
lab_results = pd.read_csv(f"{path}LABEVENTS.csv.gz")
lab_results = pd.merge(patients_sample.SUBJECT_ID,lab_results)
# lab_results.to_csv("LABEVENTS_SAMPLE.csv")


In [331]:
#roughly 20% of these items have null in the HADM ID
lab_results = lab_results.rename(columns = {'CHARTTIME':'DATE','ITEMID':'FEATURE'})
#making sure lab_results has different item_id than diagnostics. appending a LAB to the itemIDs
lab_results['FEATURE'] = lab_results['FEATURE'].apply(lambda x: f"LAB_{x}")
lab_results = lab_results[USEFUL_COLUMNS]
lab_results

,SUBJECT_ID,FEATURE,DATE,VALUE
0,4074,LAB_51279,2203-10-16 07:30:00,3.19
1,4074,LAB_51301,2203-10-16 07:30:00,8.5
2,4074,LAB_51221,2203-10-16 21:15:00,27.4
3,4074,LAB_50868,2203-10-17 05:10:00,14
4,4074,LAB_50882,2203-10-17 05:10:00,18
...,...,...,...,...
5882837,62212,LAB_51250,2176-06-12 09:50:00,88
5882838,62212,LAB_51265,2176-06-12 09:50:00,287
5882839,62212,LAB_51277,2176-06-12 09:50:00,14.5
5882840,62212,LAB_51279,2176-06-12 09:50:00,3.61


In [332]:
#take average value for lab_results
lab_results = lab_results.dropna()
#keep only numeric values, removing things like 'not done'
def is_a_number(x):
    try:
        float(x.strip())
        return True
    except:
        return False
    return True
lab_results = lab_results.loc[lab_results['VALUE'].apply(is_a_number),:]
lab_results['VALUE'] = pd.to_numeric(lab_results['VALUE'])

#TODO: filter by date

#keep average of values (can use more sophisticated methods later on)
# lab_results = lab_results.groupby(['SUBJECT_ID','FEATURE']).mean().reset_index()
lab_results

,SUBJECT_ID,FEATURE,DATE,VALUE
0,4074,LAB_51279,2203-10-16 07:30:00,3.19
1,4074,LAB_51301,2203-10-16 07:30:00,8.50
2,4074,LAB_51221,2203-10-16 21:15:00,27.40
3,4074,LAB_50868,2203-10-17 05:10:00,14.00
4,4074,LAB_50882,2203-10-17 05:10:00,18.00
...,...,...,...,...
5882837,62212,LAB_51250,2176-06-12 09:50:00,88.00
5882838,62212,LAB_51265,2176-06-12 09:50:00,287.00
5882839,62212,LAB_51277,2176-06-12 09:50:00,14.50
5882840,62212,LAB_51279,2176-06-12 09:50:00,3.61


### Microbiology Events

In [333]:
### reading in data similar to lab_results, like microbiology events
#NOTE: Does not seem to help at all
mb_events = pd.read_csv(f"{path}MICROBIOLOGYEVENTS.csv.gz")
mb_events = pd.merge(patients_sample.SUBJECT_ID,mb_events)
mb_events = mb_events[['SUBJECT_ID','AB_ITEMID','INTERPRETATION','CHARTTIME']]
mb_events.dropna(inplace= True)

#transform seperate boolean values for Resistant, intermediate and Sensitive. 
mb_events = mb_events.loc[mb_events.INTERPRETATION != 'P']

#keep only the last index in mb_events
mb_events.drop_duplicates(subset = ['SUBJECT_ID','AB_ITEMID'], inplace = True,keep = 'last')

mb_events['FEATURE'] = mb_events['AB_ITEMID'].apply(lambda x: f"MB_{int(x)}_") + mb_events.INTERPRETATION
mb_events.drop('AB_ITEMID',axis = 1,inplace = True)

mb_events.INTERPRETATION = 1
mb_events.rename(columns = {'CHARTTIME':'DATE','INTERPRETATION':'VALUE'},inplace = True)
mb_events = mb_events[USEFUL_COLUMNS]
display(mb_events)

,SUBJECT_ID,FEATURE,DATE,VALUE
2,4074,MB_90025_R,2200-09-24 01:00:00,1
3,4074,MB_90016_R,2200-09-24 01:00:00,1
5,4074,MB_90012_I,2200-09-24 01:00:00,1
6,4074,MB_90002_R,2200-09-24 01:00:00,1
20,4074,MB_90004_S,2204-01-16 04:44:00,1
...,...,...,...,...
133781,28702,MB_90004_S,2160-08-17 06:20:00,1
133782,28702,MB_90002_S,2160-08-17 06:20:00,1
133783,28702,MB_90015_R,2160-08-17 06:20:00,1
133784,28702,MB_90012_S,2160-08-17 06:20:00,1


### Admissions

In [334]:
#ETL on admissions


#feature is the admission type and value is the time spent

admissions['TIME_SPENT'] = pd.to_timedelta(pd.to_datetime(admissions.DISCHTIME)-  pd.to_datetime(admissions.ADMITTIME)).dt.total_seconds()/3600
admissions['FEATURE'] = admissions.ADMISSION_TYPE.apply(lambda x: f"ADM_{x[0:4]}")
admissions.rename(columns = {'TIME_SPENT':'VALUE','ADMITTIME': 'DATE'},inplace = True)

admissions = admissions[USEFUL_COLUMNS]

# admissions = admissions.groupby(['SUBJECT_ID','FEATURE']).aggregate('sum').reset_index()


In [335]:

admissions

,SUBJECT_ID,FEATURE,DATE,VALUE
0,4074,ADM_EMER,2200-09-15 02:08:00,281.650000
1,4074,ADM_EMER,2204-01-05 10:48:00,270.533333
2,4074,ADM_EMER,2204-02-04 07:26:00,42.233333
3,90889,ADM_EMER,2153-02-05 11:21:00,348.633333
4,72753,ADM_EMER,2129-04-05 16:40:00,69.950000
...,...,...,...,...
12656,21583,ADM_EMER,2189-06-04 22:12:00,81.883333
12657,87048,ADM_ELEC,2118-09-12 14:00:00,121.500000
12658,16761,ADM_EMER,2126-05-16 22:24:00,642.600000
12659,28702,ADM_EMER,2160-07-30 19:43:00,903.950000


## Putting it all together
This step will
- merge all the features together
- give them unique feature_ids by building a codemape
- scale the features by their maximums

In [336]:
#merging diagnosis and lab_results
# lab_results.columns = ['SUBJECT_ID','FEATURE','VALUE']
# diagnoses.columns = ['SUBJECT_ID','FEATURE','VALUE']
# mb_events.columns = ['SUBJECT_ID','FEATURE','VALUE']
# admissions.columns = ['SUBJECT_ID','FEATURE','VALUE']

features = pd.concat([lab_results,diagnoses,mb_events,admissions])

#get rid of missing values
features = features.dropna()

#keep only observation_window
# step 1: get index dates
features['DATE'] = pd.to_datetime(features['DATE'])
index_date = features[['SUBJECT_ID','DATE']].groupby(['SUBJECT_ID']).aggregate('max').reset_index()
index_date = last_times.merge(patients[['SUBJECT_ID','DOD']])
index_date.loc[index_date.DOD.notna(),'DATE'] = pd.to_datetime(index_date.DOD) - timedelta(days = PREDICTION_WINDOW)
index_date = index_date.drop('DOD',axis = 1).rename(columns = {'DATE':'INDEX_DATE'})
index_date['INDEX_DATE'] = pd.to_datetime(index_date['INDEX_DATE'])

#step 2: create function to filter on features
def process_data(features, index_date, function):
    #filter
    features = features.merge(index_date)
    features['DATE'] = pd.to_datetime(features['DATE'])
    features = features.loc[features.DATE<=features.INDEX_DATE] #lose observations above window
    features = features.loc[features.DATE>=features.INDEX_DATE-timedelta(days = OBSERVATION_WINDOW)] #lose observations before window
    features = features[['SUBJECT_ID','FEATURE','VALUE']]
    
    #aggregate
    features = features.groupby(['SUBJECT_ID','FEATURE']).aggregate(function).reset_index()
    
    return features


In [337]:
#filter and aggregate features
lab_results_processed = process_data(lab_results,index_date,np.mean)
diagnoses_processed = process_data(diagnoses,index_date,np.mean)
mb_events_processed = process_data(mb_events,index_date,np.mean)
admissions_processed = process_data(admissions,index_date,np.sum)

features =  pd.concat([lab_results_processed,diagnoses_processed,mb_events_processed,admissions_processed])

In [338]:

#scale features
max_features = features.groupby(['FEATURE']).aggregate('max').reset_index()
features = features.merge(max_features,on = 'FEATURE', suffixes=['', '_MAX'])
features['VALUE'] = features['VALUE']/features['VALUE_MAX']
features = features[['SUBJECT_ID','FEATURE','VALUE']]
features = features.dropna()
n_features = len(features.FEATURE.unique())+10
print(f"number of features: {n_features}")
display(features[0:5])

number of features: 1417


,SUBJECT_ID,FEATURE,VALUE
0,11,LAB_50802,0.100000
1,71,LAB_50802,-0.400000
2,96,LAB_50802,0.094286
3,109,LAB_50802,-0.373913
4,124,LAB_50802,0.300000


In [339]:
#create mortality table
mortality = patients_sample.copy()
mortality['DEAD'] = 1- mortality['DOD_SSN'].isna()
mortality.index = mortality.SUBJECT_ID
mortality = mortality['DEAD']
display(mortality.value_counts())
display(mortality[0:5])
mortality = mortality.to_dict()

0    7190
1    2810
Name: DEAD, dtype: int64

SUBJECT_ID
4074     1
90889    0
72753    0
64908    0
70273    0
Name: DEAD, dtype: int64

### Saving the data prior to model training
The data will be saved in an SVM_light format for reproducibility

In [340]:
# split into training and test
from sklearn.model_selection import train_test_split
train_indices,test_indices = train_test_split(features['SUBJECT_ID'].unique(), test_size = 0.2, random_state = 1)

train = features.loc[features.SUBJECT_ID.isin(train_indices)]
test = features.loc[features.SUBJECT_ID.isin(test_indices)]

In [341]:
# Following Ivan's logic, removing features which only appear in training, and features that are below a minimum threshold
feature_count = train[['FEATURE','VALUE']].groupby(['FEATURE']).aggregate('count').reset_index()
acceptable_features = feature_count.loc[feature_count.VALUE > 1, 'FEATURE']
n_features = len(acceptable_features)
train = train.loc[train.FEATURE.isin(acceptable_features),:]
test = test.loc[test.FEATURE.isin(acceptable_features),:]


In [342]:
#apply codemap to features
codemap = build_codemap(train)

# function to help output data
def create_svmlite(patient_features, mortality, output_type):
    #apply codemap
    patient_features['FEATURE'] = patient_features['FEATURE'].map(codemap)
    patient_features['VALUE'] = patient_features.VALUE.round(6)

    
    #turn patient_features into an svm_light format kinda dictionaries
    patient_features['F2V'] = list(zip(patient_features.FEATURE,patient_features.VALUE))
    patient_features = patient_features.groupby(['SUBJECT_ID'])['F2V'].apply(list)
    patient_features = patient_features.to_dict()

    patient_ids = list(patient_features.keys())
    patient_ids.sort()
    d1 = ""
    for id in patient_ids:
        patient_features[id].sort()
        features = ''
        for feature in patient_features[id]:
            features += f" {str(int(feature[0]))}:" + "{:.6f}".format(feature[1])
        if output_type == 1: d1 += f"{mortality[id]}{features} \n"
        if output_type == 2: d1 += f"{int(id)} {mortality[id]}{features} \n"

    return d1

train_file = open(f"{path}svm_light/features.train", 'wb')
bytes_written = train_file.write(bytes((create_svmlite(train, mortality, 1)), 'UTF-8'))
print(f"wrote {bytes_written} bytes for training")
test_file = train_file = open(f"{path}svm_light/features.test", 'wb')
bytes_written = train_file.write(bytes((create_svmlite(test, mortality, 1)), 'UTF-8'))
print(f"wrote {bytes_written} bytes for test")


wrote 4126614 bytes for training
wrote 993178 bytes for test


### Model Training

In [343]:
# input: Y_pred,Y_true
# output: accuracy, auc, precision, recall, f1-score
def classification_metrics(Y_pred, Y_true):
    # TODO: Calculate the above mentioned metrics
    acc = accuracy_score(Y_pred, Y_true)
    auc_ = roc_auc_score(Y_pred, Y_true)
    precision = precision_score(Y_pred, Y_true)
    recall = recall_score(Y_pred, Y_true)
    f1score = f1_score(Y_pred, Y_true)
    # NOTE: It is important to provide the output in the same order
    return acc, auc_, precision, recall, f1score
# input: Name of classifier, predicted labels, actual labels
def display_metrics(classifierName, Y_pred, Y_true):
    print("______________________________________________")
    print(("Classifier: " + classifierName))
    acc, auc_, precision, recall, f1score = classification_metrics(Y_pred, Y_true)
    print(("Accuracy: " + str(acc)))
    print(("AUC: " + str(auc_)))
    print(("Precision: " + str(precision)))
    print(("Recall: " + str(recall)))
    print(("F1-score: " + str(f1score)))
    print("______________________________________________")
    print("")

# input: X_train, Y_train and X_test
# output: Y_pred
def logistic_regression_pred(X_train, Y_train, X_test):
    log_model = LogisticRegression(random_state=1)
    log_model.fit(X_train, Y_train)
    Y_pred = log_model.predict(X_test)
    return Y_pred

def random_forest_pred(X_train, Y_train, X_test):
    model = RandomForestClassifier(random_state=1)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    return Y_pred

X_train, Y_train = load_svmlight_file(f"{path}svm_light/features.train", n_features=n_features)
X_test, Y_test = load_svmlight_file(f"{path}svm_light/features.test", n_features=n_features)


### Results

In [344]:
from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
display_metrics("Logistic Regression", logistic_regression_pred(X_train, Y_train, X_test), Y_test)
# display_metrics("SVM", svm_pred(X_train, Y_train, X_test), Y_test)
display_metrics("Random Forest Tree", random_forest_pred(X_train, Y_train, X_test), Y_test)


/Users/hassan/opt/anaconda3/envs/Homework1/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


______________________________________________
Classifier: Logistic Regression
Accuracy: 0.8826318909306461
AUC: 0.7977444126967013
Precision: 0.5551601423487544
Recall: 0.6812227074235808
F1-score: 0.611764705882353
______________________________________________

______________________________________________
Classifier: Random Forest Tree
Accuracy: 0.8772969768820391
AUC: 0.8044381642622178
Precision: 0.4412811387900356
Recall: 0.7126436781609196
F1-score: 0.545054945054945
______________________________________________

